### Homework 3:
### Name: Daniel Jang
### GitHub: hjang8659
### USC ID: 6894-6426-04
### Due: Feb 23, 2024

In [1]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np
import math
import re
import os
import glob

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Modeling
import statsmodels.api as sm

# Machine Learning Algorithms and Model Building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import DistanceMetric
from scipy.linalg import pinv
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils import resample

# Feature Scaling and Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif

# Model Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
from sklearn.model_selection import cross_val_score

# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

# Deep Learning
import tensorflow as tf
from tensorflow import keras

c:\Users\hjang\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### (1) Time Series Classification Part 1: Feature Creation/Extraction
### (a)
#### Excel was used to combine datasets and for data cleaning, as authorized by posts on Piazza and homework's footnote.

In [2]:
bend1 = pd.read_excel("../data/bending1/combined.xlsx")
bend2 = pd.read_excel("../data/bending2/combined.xlsx")
cycle = pd.read_excel("../data/cycling/combined.xlsx")
lying = pd.read_excel("../data/lying/combined.xlsx")
sit = pd.read_excel("../data/sitting/combined.xlsx")
stand = pd.read_excel("../data/standing/combined.xlsx")
walk = pd.read_excel("../data/walking/combined.xlsx")

### (b)

In [3]:
test_dfs = []
train_dfs = []

for df, activity in zip([bend1, bend2, cycle, lying, sit, stand, walk],
                        ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']):
    test_df = df[df['Source.Name'].isin(['dataset1.csv', 'dataset2.csv'])].copy()
    test_df['activity'] = activity
    test_dfs.append(test_df)

    train_df = df[~df['Source.Name'].isin(['dataset1.csv', 'dataset2.csv'])].copy()
    train_df['activity'] = activity
    train_dfs.append(train_df)

test = pd.concat(test_dfs, ignore_index=True)
train = pd.concat(train_dfs, ignore_index=True)

In [4]:
train

,Source.Name,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,dataset3.csv,0,42.00,0.71,21.25,0.43,30.00,0.00,bending1
1,dataset3.csv,250,41.50,0.50,20.25,1.48,31.25,1.09,bending1
2,dataset3.csv,500,41.50,0.50,14.25,1.92,33.00,0.00,bending1
3,dataset3.csv,750,40.75,0.83,15.75,0.43,33.00,0.00,bending1
4,dataset3.csv,1000,40.00,0.71,20.00,2.74,32.75,0.43,bending1
...,...,...,...,...,...,...,...,...,...
35514,dataset9.csv,118750,31.50,1.66,12.50,3.20,14.25,4.44,walking
35515,dataset9.csv,119000,27.33,1.25,11.33,0.94,20.00,4.00,walking
35516,dataset9.csv,119250,37.80,7.68,14.20,2.48,17.25,0.83,walking
35517,dataset9.csv,119500,33.75,1.30,15.75,5.21,16.50,2.69,walking


In [5]:
test

,Source.Name,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,dataset1.csv,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,dataset1.csv,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1
2,dataset1.csv,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1
3,dataset1.csv,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1
4,dataset1.csv,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...,...,...
6715,dataset2.csv,118750,27.75,5.76,13.25,1.30,16.25,2.49,walking
6716,dataset2.csv,119000,38.25,5.12,14.75,2.17,15.00,6.36,walking
6717,dataset2.csv,119250,34.00,2.83,17.00,5.39,9.00,4.97,walking
6718,dataset2.csv,119500,29.00,5.72,12.00,6.60,12.67,5.44,walking


### (c)
### i.
### Standard deviation, kurtosis, first quartile, third quartile, median, entropy, autocorrelation.
### ii.

In [6]:
columns_to_calculate = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

aggregation_functions = {
    'avg_rss12': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'var_rss12': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'avg_rss13': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'var_rss13': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'avg_rss23': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'var_rss23': ['min', 'max', 'mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
}

result_names = [
    'min1', 'max1', 'mean1', 'median1', 'std1', '1st_quart1', '3rd_quart1',
    'min2', 'max2', 'mean2', 'median2', 'std2', '1st_quart2', '3rd_quart2',
    'min3', 'max3', 'mean3', 'median3', 'std3', '1st_quart3', '3rd_quart3',
    'min4', 'max4', 'mean4', 'median4', 'std4', '1st_quart4', '3rd_quart4',
    'min5', 'max5', 'mean5', 'median5', 'std5', '1st_quart5', '3rd_quart5',
    'min6', 'max6', 'mean6', 'median6', 'std6', '1st_quart6', '3rd_quart6'
]

result_df = train.groupby(['activity', 'Source.Name'])[columns_to_calculate].agg(aggregation_functions)

result_df.columns = result_df.columns.map(''.join)
result_df.columns = result_names
train_extracted = result_df.reset_index(drop=True)
train_extracted

,min1,max1,mean1,median1,std1,1st_quart1,3rd_quart1,min2,max2,mean2,...,std5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std6,1st_quart6,3rd_quart6
0,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
1,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
2,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
3,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,2.488862,22.2500,24.00,0.0,5.26,0.679646,0.500,0.622534,0.43,0.8700
4,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,3.318301,20.5000,23.75,0.0,2.96,0.555312,0.490,0.487826,0.00,0.8300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
70,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
71,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
72,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


### iii.

In [7]:
def bootstrap_std_ci(data, alpha=0.1, n_iterations=1000):
    original_std = np.std(data)
    bootstrap_samples = [np.random.choice(data, size=len(data), replace=True) for _ in range(n_iterations)]
    bootstrap_std = np.std(bootstrap_samples, axis=1)
    ci_low = np.percentile(bootstrap_std, (alpha / 2) * 100)
    ci_high = np.percentile(bootstrap_std, (1 - alpha / 2) * 100)

    return original_std, ci_low, ci_high

results = {}

for column in train_extracted.columns:
    original_std, ci_low, ci_high = bootstrap_std_ci(train_extracted[column])
    results[column] = {'original_std': original_std, 'ci_low': ci_low, 'ci_high': ci_high}

for column, result in results.items():
    print(f"Feature: {column}")
    print(f"Original Standard Deviation: {result['original_std']:.4f}")
    print(f"Bootstrap CI (90%): [{result['ci_low']:.4f}, {result['ci_high']:.4f}]")
    print()

Feature: min1
Original Standard Deviation: 9.0804
Bootstrap CI (90%): [7.7930, 10.3329]

Feature: max1
Original Standard Deviation: 4.2681
Bootstrap CI (90%): [3.1510, 5.1883]

Feature: mean1
Original Standard Deviation: 4.9305
Bootstrap CI (90%): [4.3503, 5.4260]

Feature: median1
Original Standard Deviation: 4.9921
Bootstrap CI (90%): [4.4149, 5.4878]

Feature: std1
Original Standard Deviation: 1.7694
Bootstrap CI (90%): [1.5471, 1.9590]

Feature: 1st_quart1
Original Standard Deviation: 5.7864
Bootstrap CI (90%): [5.2032, 6.2514]

Feature: 3rd_quart1
Original Standard Deviation: 4.7107
Bootstrap CI (90%): [3.8991, 5.4104]

Feature: min2
Original Standard Deviation: 0.0000
Bootstrap CI (90%): [0.0000, 0.0000]

Feature: max2
Original Standard Deviation: 5.1807
Bootstrap CI (90%): [4.7012, 5.5253]

Feature: mean2
Original Standard Deviation: 1.5914
Bootstrap CI (90%): [1.4120, 1.7248]

Feature: median2
Original Standard Deviation: 1.4314
Bootstrap CI (90%): [1.2636, 1.5559]

Feature: st

### iv.
### In selecting the top 3 most important time-domain features from the provided data, the focus is on features with both high stds and relatively narrow bootstrap CIs. Min stands out with the highest stds of 9.0804 and a reasonably narrow CI of [7.8645, 10.2210]. Additionally, max and 3rd quarter (75%) are noteworthy, having stds of 5.4256 and 4.7107, with bootstrap CIs [4.2531, 6.2634] and [3.9382, 5.4659]. These features are selected based on their potential to capture significant variability in the data while maintaining stability in the estimation process.
### 2. ISLR 3.7.4
#### (a) Adding extra terms like X^2 and X^3 in cubic regression allows the model to closely fit the training data, providing more flexibility. Therefore, the training RSS for cubic regression is likely to be lower than for SLR. This is because the cubic regression captures both linear and non-linear relationships, while SLR only captures linear relationships. The added flexibility in cubic regression enables it to minimize the training RSS by fitting the data more closely.
#### (b) If the actual relationship between X and Y is linear, SLR tends to generalize better to new data, resulting in a lower test RSS. For the cubic model, it might overfit the training data and show a higher test RSS with its additional complexity. The cubic regression, having been trained on both linear and non-linear patterns, might not generalize as well to new data with a predominantly linear relationship.
#### (c) When the true relationship is not linear, cubic regression, with its ability to capture non-linear patterns, should have a lower training RSS compared to linear regression. This is because the cubic regression model can better adapt to the underlying non-linearities in the data during training, providing a more accurate representation of the true relationship.
#### (d) For the test RSS, it all depends on how close the true relationship f(x) is to being either linear or cubic. If f(x) is more linear, SLR is expected to perform better, yielding a lower test RSS. If f(x) is closer to cubic, SLR might struggle to capture the complexity, resulting in a higher test RSS. The test RSS reflects the model's ability to generalize to new data, and in cases where the true relationship is predominantly linear, the simpler SLR is likely to outperform the cubic regression in terms of test performance.